# Premier League 2024-25 Data Analysis - Part 1: Exploration & Cleaning

This notebook focuses on the initial exploration and cleaning of the Premier League 2024-25 dataset. We'll explore the data structure, handle missing values, and prepare it for further analysis.

# Premier League 2024-25 Data Analysis - Part 1: Exploration & Cleaning

This notebook focuses on the initial exploration and cleaning of the Premier League 2024-25 dataset. We'll explore the data structure, handle missing values, and prepare it for further analysis.

## 1. Import Libraries

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Display settings
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

## 2. Load the Data

In [ ]:
# Load the Premier League dataset
file_path = '../data/fbref_PL_2024-25.csv'
df = pd.read_csv(file_path)

# Check if the file exists and has data
print(f"Dataset shape: {df.shape}")
print(f"Number of columns: {len(df.columns)}")

# Display the first few rows of the dataset
df.head()

## 3. Understanding the Data Structure

In [ ]:
# Get information about the dataset
print("Dataset Info:")
df.info()

In [ ]:
# Get summary statistics
df.describe().T

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
})

# Display columns with missing values
missing_df[missing_df['Missing Values'] > 0].sort_values('Missing Values', ascending=False)

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

## 4. Explore Categorical Variables

In [ ]:
# Identify categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns: {categorical_columns}")

# Let's explore the distribution of some key categorical variables
for col in categorical_columns[:5]:  # Limit to first 5 to avoid overwhelming output
    print(f"\n{col} - Unique values: {df[col].nunique()}")
    print(df[col].value_counts().head(10))

## 5. Explore Numerical Variables

In [ ]:
# Identify numerical columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Numerical columns: {len(numerical_columns)}")

# Let's create histograms for some key numerical variables
key_numerical = numerical_columns[:5]  # Select first 5 for visualization

plt.figure(figsize=(15, 10))
for i, col in enumerate(key_numerical):
    plt.subplot(2, 3, i+1)
    sns.histplot(df[col].dropna(), kde=True)
    plt.title(f'Distribution of {col}')
    plt.tight_layout()
plt.show()

## 6. Data Cleaning and Preparation

In [ ]:
# Create a copy of the original dataframe
df_clean = df.copy()

# Handle missing values
# For numerical columns: fill with median or mean
# For categorical columns: fill with mode or 'Unknown'

# First, let's examine the columns with missing values to make informed decisions
columns_with_missing = missing_df[missing_df['Missing Values'] > 0].index.tolist()
print(f"Columns with missing values: {columns_with_missing}")

In [ ]:
# Implementation of cleaning logic for our dataset

# 1. Convert data types
# Convert numerical columns that might be stored as strings
numeric_cols = ['Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 
                'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG']

for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# 2. Handle missing values
# For numerical columns: fill with median
for col in df_clean.select_dtypes(include=['int64', 'float64']).columns:
    if df_clean[col].isnull().sum() > 0:
        median_val = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna(median_val)
        print(f"Filled {col} missing values with median: {median_val}")

# For categorical columns: fill with 'Unknown'
for col in df_clean.select_dtypes(include=['object']).columns:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna('Unknown')
        print(f"Filled {col} missing values with 'Unknown'")

# 3. Handle duplicates
dupe_count = df_clean.duplicated().sum()
if dupe_count > 0:
    df_clean = df_clean.drop_duplicates()
    print(f"Removed {dupe_count} duplicate rows")
else:
    print("No duplicates found")

# 4. Clean up text data
# Example: Standardize team names if needed
if 'Squad' in df_clean.columns:
    # Print unique values to check if standardization is needed
    print("\nUnique team names:")
    print(df_clean['Squad'].unique())

# 5. Identify and handle outliers
# Example: Check for outliers in the Age column
if 'Age' in df_clean.columns:
    q1 = df_clean['Age'].quantile(0.25)
    q3 = df_clean['Age'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    outliers = df_clean[(df_clean['Age'] < lower_bound) | (df_clean['Age'] > upper_bound)]
    print(f"\nNumber of age outliers: {len(outliers)}")
    if len(outliers) > 0:
        print("Age outliers:")
        print(outliers[['Player', 'Age', 'Squad']].head())

# Print the shape after cleaning
print(f"\nDataset shape after cleaning: {df_clean.shape}")

## 7. Feature Engineering

In [ ]:
# Add new features that might be useful for analysis
df_clean = df.copy()

# Create goals per 90 minutes
df_clean['goals_per_90'] = df_clean['Gls'] / df_clean['90s']

# Create assists per 90 minutes
df_clean['assists_per_90'] = df_clean['Ast'] / df_clean['90s']

# Create goal contributions (G+A) per 90 minutes
df_clean['goal_contributions_per_90'] = df_clean['G+A'] / df_clean['90s']

# Calculate minutes per goal
df_clean['minutes_per_goal'] = df_clean['Min'] / df_clean['Gls'].replace(0, np.nan)

# Calculate goal conversion rate (goals divided by expected goals)
df_clean['goal_conversion'] = df_clean['Gls'] / df_clean['xG']

# Calculate assist conversion rate (assists divided by expected assisted goals)
df_clean['assist_conversion'] = df_clean['Ast'] / df_clean['xAG']

# Calculate minutes played percentage (based on total possible minutes)
# Assuming a full season has 38 games * 90 minutes = 3420 minutes
df_clean['minutes_percentage'] = (df_clean['Min'] / 3420) * 100

# Calculate starts percentage
df_clean['starts_percentage'] = (df_clean['Starts'] / df_clean['MP']) * 100

# Handle infinity and NaN values from divisions by zero
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)

# Display the new features
print("New features created:")
new_features = ['goals_per_90', 'assists_per_90', 'goal_contributions_per_90', 
               'minutes_per_goal', 'goal_conversion', 'assist_conversion',
               'minutes_percentage', 'starts_percentage']
df_clean[new_features].head()

## 8. Save Cleaned Dataset

In [ ]:
# Save the cleaned dataset
df_clean.to_csv('../data/pl_2024_25_cleaned.csv', index=False)
print("Cleaned dataset saved successfully!")

## 9. Preliminary Visualizations

In [ ]:
# Create some initial visualizations to understand the data better
# These will be expanded in the next notebook focused on visualization

# Example visualization code (to be modified based on actual data):
# plt.figure(figsize=(12, 8))
# sns.heatmap(df_clean[numerical_columns[:10]].corr(), annot=True, cmap='coolwarm')
# plt.title('Correlation Matrix of Key Numerical Features')
# plt.tight_layout()
# plt.show()

## 10. Next Steps

In the next notebook, we'll:
1. Perform in-depth statistical analysis
2. Create comprehensive visualizations
3. Identify patterns and trends in the data
4. Prepare for predictive modeling

## 13. Data Dictionary

Based on our exploration, here's an explanation of the key columns in our dataset:

- **Rk**: Rank or index number
- **Player**: Player's name
- **Nation**: Player's nationality
- **Pos**: Player's position (DF: Defender, MF: Midfielder, FW: Forward)
- **Squad**: The team/club the player belongs to
- **Age**: Player's age
- **Born**: Year of birth
- **MP**: Matches played
- **Starts**: Number of matches started
- **Min**: Minutes played
- **90s**: Number of 90-minute periods played (Minutes played / 90)
- **Gls**: Goals scored
- **Ast**: Assists
- **G+A**: Goals + Assists
- **G-PK**: Non-penalty goals
- **PK**: Penalty kicks scored
- **PKatt**: Penalty kicks attempted
- **CrdY**: Yellow cards
- **CrdR**: Red cards
- **xG**: Expected goals
- **npxG**: Non-penalty expected goals
- **xAG**: Expected assisted goals
- **PrgC**: Progressive carries
- **PrgP**: Progressive passes
- **PrgR**: Progressive passes received

The dataset appears to contain player-level statistics from the English Premier League 2024-25 season.